In [1]:
%reload_ext autoreload
%autoreload 2

import time
from tqdm import tqdm
import numpy as np
import scipy as sp
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.transforms import v2 as transforms

from matplotlib import pyplot as plt

from networkAlignmentAnalysis.models.registry import get_model
from networkAlignmentAnalysis.datasets import get_dataset
from networkAlignmentAnalysis.experiments.registry import get_experiment
from networkAlignmentAnalysis import utils
from networkAlignmentAnalysis import files
from networkAlignmentAnalysis import train

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', DEVICE)

using device:  cpu


In [3]:
# TODO
# 1.1. include additional AlignmentModel methods stored in extra class in base model

# 4. Rewrite existing analysis pipelines
# 5. SLURM!!!!

# Figure out why convolutional alignment measurement is slow...
# still working on if it's possible to speed up measure_alignment for convolutional layers

# Basic alignment_comparison Analyses (or maybe for alignment_stats):
# - compare initial to final alignment...
# - compare initial alignment to delta weight norm...
# - observe alignment of delta weight
# - compare alignment to outgoing delta weight norm!

# Eigenfeature analyses:
# - Measure beta_adversarial (figure out how adversarial examples map onto eigenvectors)
# - Determine contribution of each eigenfeature on performance with a eigenvector dropout experiment
# ------ start by just looking at amplitude of activity on each eigenvector within each layer

# alignmentShaping.ipynb has an adversarial experiment worth looking at

In [2]:
from networkAlignmentAnalysis.experiments.alignment_stats import AlignmentStatistics
from argparse import ArgumentParser


In [4]:
expmt = AlignmentStatistics(args=['--network', 'AlexNet', '--epochs', '10', '--replicates', '2', '--use_prev'])


# load networks 
nets, optimizers = expmt.load_networks()

# load dataset
dataset = get_dataset(expmt.args.dataset,
                      build=True,
                      transform_parameters=nets[0],
                      device=expmt.args.device)

# train networks
train_results, test_results = expmt.train_networks(nets, optimizers, dataset)

Namespace(network='AlexNet', dataset='MNIST', optimizer='Adam', batch_size=1024, default_lr=0.001, default_dropout=0, default_wd=0, num_drops=9, dropout_by_layer=False, epochs=10, replicates=2, use_flag=False, use_prev=True, save_ckpts=False, nosave=False, justplot=False, save_networks=False, showprms=False, showall=False, device=None, use_timestamp=False, timestamp=None)
loaded networks from previous checkpoint
training networks...
resuming training from checkpoint on epoch 10


training epoch: 0it [00:00, ?it/s]


testing networks...


  0%|          | 0/10 [00:00<?, ?it/s]/Users/celiaberon/miniforge3/envs/networkAlignmentAnalysis/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/Users/celiaberon/miniforge3/envs/networkAlignmentAnalysis/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the r

In [34]:
dropout_parameters = dict(num_drops=expmt.args.num_drops, by_layer=expmt.args.dropout_by_layer)
dropout_results = train.progressive_dropout(nets, dataset, alignment=test_results['alignment'], **dropout_parameters)

In [10]:
model_name = 'MLP'
dataset_name = 'MNIST'

net = get_model(model_name, build=True, dataset=dataset_name).to(DEVICE)
dataset = get_dataset(dataset_name, build=True, transform_parameters=net, device=DEVICE)

init_test = train.test([net], dataset)
print(init_test['accuracy'])

optim = torch.optim.Adam(net.parameters(), lr=1e-2)
prms = dict(num_epochs=100, alignment=False, average_correlation=False)
_ = train.train([net], [optim], dataset, **prms)

post_test = train.test([net], dataset)
print(post_test['accuracy'])

beta, evals, evecs = net.measure_eigenfeatures(dataset.test_loader, with_updates=False)

100%|██████████| 10/10 [00:03<00:00,  2.66it/s]


[tensor(10.2166, device='cuda:0')]


100%|██████████| 10/10 [00:00<00:00, 22.55it/s]


[tensor(95.1885, device='cuda:0')]


In [16]:
# Next Steps: 
# - write a method to go through a dataloader and measure the contribution of each output onto the eigenvectors
# (e.g. the average for classes)

In [ ]:
# out[i][j][k] = input[index[i][j][k]][j][k]  # if dim == 0
# out[i][j][k] = input[i][index[i][j][k]][k]  # if dim == 1
# out[i][j][k] = input[i][j][index[i][j][k]]  # if dim == 2

# out = (num_classes, num_dims, num_samples)


In [28]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(50, 10)
    
    def forward(self, x):
        return self.layer1(x)
    
net = model()
input = torch.normal(0, 1, (1000, 50))
output = net(input)
output.shape

torch.Size([1000, 10])

In [29]:
next(net.parameters()).device.type

'cpu'